<a href="https://colab.research.google.com/github/leticiafcarvalho/cats_and_dogs/blob/main/cats_and_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Conectar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Bibliotecas necessárias
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
import numpy as np
import os

In [5]:
# Caminho das pastas no Drive
base_dir = '/content/drive/My Drive/cats_and_dogs'  # Ajuste conforme o local das suas pastas

In [6]:
# Criar Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Divisão treino/validação
)

In [7]:
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Found 1619 images belonging to 2 classes.


In [8]:
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 404 images belonging to 2 classes.


In [9]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Congelar os pesos da base


<ipython-input-9-0bfd68664fbf>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(150, 150, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Duas classes: 0 ou 1
])

In [11]:
# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Treinar o modelo
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


51/51 ━━━━━━━━━━━━━━━━━━━━ 260s 4s/step - accuracy: 0.8296 - loss: 0.3764 - val_accuracy: 0.9109 - val_loss: 0.2284
Epoch 2/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - accuracy: 0.9353 - loss: 0.1541 - val_accuracy: 0.9158 - val_loss: 0.2300
Epoch 3/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.9544 - loss: 0.1132 - val_accuracy: 0.9208 - val_loss: 0.2000
Epoch 4/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.9482 - loss: 0.1199 - val_accuracy: 0.9183 - val_loss: 0.2094
Epoch 5/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.9378 - loss: 0.1466 - val_accuracy: 0.9059 - val_loss: 0.2250
Epoch 6/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 91s 1s/step - accuracy: 0.9499 - loss: 0.1235 - val_accuracy: 0.9307 - val_loss: 0.1802
Epoch 7/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.9488 - loss: 0.1198 - val_accuracy: 0.9307 - val_loss: 0.1890
Epoch 8/10
43/51 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.9654 - loss: 0.0886

In [ ]:
# Avaliar o modelo
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.2f}")


In [ ]:
# Salvar o modelo
model_path = '/content/drive/My Drive/cats_dogs_model.h5'  # Caminho para salvar no Drive
model.save(model_path)
print(f"Modelo salvo em: {model_path}")


In [ ]:
# Testar o modelo com uma imagem de exemplo
def predict_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    class_label = "Dog" if prediction[0] > 0.5 else "Cat"
    print(f"Prediction: {class_label} (Confidence: {prediction[0][0]:.2f})")

In [ ]:
# Substitua pelo caminho da imagem que deseja testar
test_image_path = '/content/drive/My Drive/maltese-1123016_1280.jpg'  # Caminho da imagem de teste
predict_image(test_image_path)